In [90]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os
import argparse
import random
import torch
import torchaudio
import torchvision.io
import pyloudnorm as pyln
import glob
from matplotlib import pyplot as plt
from IPython.display import Video
from IPython.display import Audio
from torchvision.transforms import v2


In [14]:
rate = 16000
table = pq.read_table(os.path.join('..', 'data', 'lipsync.parquet')).to_pandas()

In [15]:
from IPython.display import Audio
Audio(table['audio'][0], rate=rate)

In [55]:
audio = np.ascontiguousarray(torch.Tensor(table['audio'][0].copy()).reshape(1, -1).expand(2, -1).numpy())
Audio(audio, rate=rate)

In [179]:
sprites = v2.Resize((256 * 4, 256 * 4))(torchvision.io.decode_image('../images/demolipssheet_bg.png', 'RGB')).permute([1, 2, 0])
# Permute goes from CHW order to HWC
# sprites will be 4x4 grid of 256x256 sprites

In [180]:
viseme_labels = ['Ah', 'D', 'Ee', 'F', 'L', 'M', 'Neutral', 'Oh', 'R', 'S', 'Uh', 'Woo']

visemes = {
    'Ah': [0, 1],
    'D': [0, 3],
    'Ee': [0, 2],
    'F': [1, 3],
    'L': [1, 1],
    'M': [2, 0],
    'Neutral': [1, 0],
    'Oh': [1, 2],
    'R': [2, 3],
    'S': [2, 1],
    'Uh': [2, 2],
    'Woo': [0, 0],
}

In [181]:
for frame in range(v.shape[0]):
    vi = viseme_labels[table['visemes'][0][frame]]
    pos = visemes[vi]
    v[frame, :, :, :] = sprites2[pos[0] * 256 : pos[0] * 256 + 256, pos[1] * 256 : pos[1] * 256 + 256, :]
torchvision.io.write_video('out.mp4', v, fps=30, audio_array=audio, audio_fps=rate, audio_codec='aac')
Video('out.mp4')

In [183]:
table['visemes'][0].shape

(101,)

In [194]:
def make_video(audio, rate, vis_indexes):
    frames = vis_indexes.shape[0]
    # Make copy of audio in stereo contiguous format for writing to video
    a = np.ascontiguousarray(torch.Tensor(audio.copy()).reshape(1, -1).expand(2, -1).numpy())
    v = torch.zeros(frames, 256, 256, 3)
    for i in range(frames):
        vi = viseme_labels[vis_indexes[i]]
        pos = visemes[vi]
        v[i, :, :, :] = sprites[pos[0] * 256 : pos[0] * 256 + 256, pos[1] * 256 : pos[1] * 256 + 256, :]
    torchvision.io.write_video('out.mp4', v, fps=30, audio_array=a, audio_fps=rate, audio_codec='aac')

make_video(table['audio'][205], rate, table['visemes'][205])
Video('out.mp4')